# Moody's Risk Calc-API Data Input
The purpose of this program is the replace the manual scoring process currently implemented into a more automated process. This will enable the end user more ease and more accuracy in creating the monthly scores for all new and exiting principals, both currently and all previous months.¶

In [1]:
import sharepy, getpass, json, os, pandas as pd, numpy as np
from datetime import datetime
pd.set_option("display.max_columns", None)

## Step 1: Data Call
This portion of the process is intended to connect this program to the local file where the raw data is stored. Please refer to the README documentation in the GRBT File for further information.  

In [3]:
#since all 6 datasets must be treated in the same manner, i will load all 6 and run through the same process concurrently
project_root='C:/Users/n1564120/Downloads'
Stated='Stated'
Adjusted='Adjusted'
comst=pd.read_csv(project_root+ '/com_st_042022.csv')
comst.insert(loc=24, column='Pilot?', value=Stated)
comad=pd.read_csv(project_root+'/com_ad_042022.csv')
comad.insert(loc=24, column='Pilot?', value=Adjusted)
const=pd.read_csv(project_root+'/con_st_042022.csv')
const.insert(loc=24, column='Pilot?', value=Stated)
conad=pd.read_csv(project_root+'/con_ad_042022.csv')
conad.insert(loc=24, column='Pilot?', value=Adjusted)
inad=pd.read_csv(project_root+'/in_ad_042022.csv')
inad.insert(loc=24, column='Pilot?', value=Adjusted)
inst=pd.read_csv(project_root+'/in_st_042022.csv')
inst.insert(loc=24, column='Pilot?', value=Stated)

#ref/lookup table file, not to be merged
#mp=pd.read_csv('C:/Users/n1564120/OneDrive - Liberty Mutual/Documents/mappingtable.csv', header=0)

###  Combine the 6 GIBSS files

In [4]:
#create the consolidated FAW file, beginning with the common element columns
df = comst.append([comad, const, conad, inad, inst])

### Create Date Fields for Common Data Elements

In [5]:
#complete the statement date and current date common element columns

df['Datadate']=pd.to_datetime(df['Datadate'], format='%m/%d/%Y')
df['Statement Date - Year']=df['Datadate'].dt.year
df['Statement Date - Month']=df['Datadate'].dt.month
currentDatetime=datetime.now()
date=currentDatetime.date()
df['Current Date - Year']=date.strftime('%Y')
df['Current Date - Month']=date.strftime('%m')
df['Prior Year']=(df['Statement Date - Year']-1)


In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
df.info(verbose=True, null_counts=True)
df.tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47684 entries, 0 to 8788
Data columns (total 260 columns):
 #   Column                                                                  Non-Null Count  Dtype         
---  ------                                                                  --------------  -----         
 0   Account ID                                                              47529 non-null  float64       
 1   Account Name                                                            47529 non-null  object        
 2   Principal ID                                                            47684 non-null  int64         
 3   Principal Name                                                          47684 non-null  object        
 4   Principal that Represents the Account                                   47529 non-null  float64       
 5   Account Office                                                          47529 non-null  object        
 6   Datadate              

,Account ID,Account Name,Principal ID,Principal Name,Principal that Represents the Account,Account Office,Datadate,Country,Currency,Units,Statement Period,Statement Type,Template Type,FAW ID,Statement Date - Year,Statement Date - Month,Current Date - Year,Current Date - Month,Industry Classification,Industry Definition,User-Defined Time Period (optional),Financial Statement Only EDF,Assets,Current Assets,Pilot?,Cash,Accounts Receivable,Inventory,Goodwill,Fixed Assets,Liabilities,Current Liabilities,Accounts Payable,Long Term Liabilities,Long Term Debt and Notes Payable,Other Provisions,Shareholders' Equity,Retained Earnings,Sales,Cost of Sales,Gross Profit Margin,"Earnings Before Interest, Taxes, Depreciation and Amortization",Earnings Before Interest and Taxes (EBIT),Interest Income,Interest Expense (-),Net Income (Loss),Depreciation and Amortization - ADJUSTMENTSTORECONCILENETINCOME,Intangibles,Marketable Securities,Long Term Debt Due Within One Year,Other Short Term Interest Bearing Debt,Cash Flow from Operating Activities,Net Income (Loss)1,Capital Expenditures,Free Cash Flow,Fixed Charge Coverage,Interest Bearing Debt,EBITDA,Return on Capital (%),Return on Equity (%),Interest Bearing Debt to Shareholders' Equity,Return on Assets (%),Gross Profit (Loss) to Sales (%),G&A Expense to Sales (%),Net Profit (Loss) to Sales (%),EBITDA to Sales,Interest Bearing Debt to EBITDA,EBITDA to Interest Expense,Interest Bearing Debt to Operating Cash Flow,Cash Flow to Interest Expense,Free Cash Flow to Interest Expense,"Selling, General and Administrative Expenses",Depreciation and Amortization,Other Income (Expense) and Interest,Cash Flow from Investing Activities,Cash Flow from Financing Activities,Proceeds from Issuance of Debt,Issuance of Common Shares,Purchase of Convertible Preferred Securities,Purchase of Common Shares,Payments on Debt,Payments on Capital Lease Obligations,Dividend Payments,Consolidated Id,Product Segment,ProbableGrossExposureAmt,CDs and Marketable Securities,Accounts Receivable - Trade,Accounts Receivable - Other,Accounts and Notes Receivable - Related Parties,Accounts and Notes Receivable - Stockholders',Accounts and Notes Receivable - Officers and Employees,Retainage,Cost and Estimated Earnings in Excess of Billings,Accounts Receivable - Trade.1,Accounts Receivable - Over 90 Days,Retainage.1,Accounts Receivable - Other.1,Accounts and Notes Receivable - Related Parties.1,Accounts and Notes Receivable - Stockholders'.1,Accounts and Notes Receivable - Officers and Employees.1,Other Intangible Assets,Equipment,Buildings,Other Fixed Assets,Real Estate,Working Capital Line,Revolving Line of Credit,Bank Overdraft,Notes Payable - Other,Current Portion - Equipment Debt,Accounts Payable - Trade,Retainage.2,Billings in Excess of Cost and Earned Profit,Accounts Payable - Related Parties,Loans and Payables - Stockholders',Accounts Payable - Other,Revolving Line of Credit.1,Accounts and Notes Payable - Related Parties,Equipment Debt,Accounts and Notes Payable - Stockholders',Notes Payable - Other.1,Real Estate Debt,Other Interest Bearing Debt,Other Long-Term Debt (Non-Interest Bearing),Revenues,Cost of Revenues,Gross Profit,Operating Income,Interest Expense,Net Income Before Taxes,Net Income After Taxes,"Net Income After Taxes, Distributions, and Dividends",Cash flow from Operations,Max Backlog,WC % Case,NW % Case,Interest Coverage,Current Ratio,Quick Ratio,True Cash,Underbillings to Shareholders' Equity (%),Underbillings to Working Capital (%),Liquidity to Working Capital (%),Liquidity to Overbillings (%),Accounts Receivable - Turnover,Accounts Payable - Turnover,Gross Profit (%),G&A / Revenues (%),Net Profit (%),ROE,Debt/Worth,Overhead to Shareholders' Equity (%),Operating Profit (%),G&A Expenses,Dividends and Distributions,Dividends,Subchapter - S Distributions,Investments,Financing,Capital Lease Obligations - Short Term,Operating Lease Obligations - Short Term,Current Portion - Other,Capital Lease Obligations - Long Term,O

## Run Initial Filters

In [7]:

df1=df[(df['Product Segment']=='Core Commercial') | (df['Product Segment']=='Core Contract')|
        (df['Product Segment']=='Specialty Commercial')|(df['Product Segment']=='Specialty Contract')]

#df2=pd.concat([df1,dfcc]).drop_duplicates().reset_index(drop=True)

df14=df1[df1['Statement Type'].isin(['Review', 'Qualified Audit', 'Unqualified Audit'])]

#remove entries where statement period DNE 12
df15=df14[(df14['Statement Period'] == 12)]



In [8]:
df39=(df15.loc[df15['Account Office'].isin(['Minneapolis', 'Chicago', 'New York', 'Dallas', 'Atlanta',
       'Philadelphia', 'Kansas City', 'San Francisco', 'Los Angeles',
       'Seattle', 'Orlando', 'Baltimore', 'Wausau', 'Boston',
       'Louisville', 'Charlotte', 'Warrenville', 'Syracuse', 'Cincinnati',
       'Phoenix', 'Hoffman Estates', 'National Bond Center', 'Pittsburgh',
       'Houston', 'Oklahoma City', 'Harrisburg', 'Jackson', 'Home Office',
       'Denver', 'St. Louis', 'St.Pete', 'Vista', 'San Diego',
        'Detroit', 'Irvine', 'ICU', 'Miami - US'])])
df40=(df39.loc[df39['Product Segment']=='Core Contract'])
dfmerge=(df15[~df15['Principal ID'].isin(df40['Principal ID'])])
dfmerge["Combined"] = dfmerge["Pilot?"].str.cat(dfmerge[["Principal ID", "Datadate", 'Template Type', 'Statement Type', 'FAW ID']].astype(str), sep="_")
#(dfmerge.loc[dfmerge._merge=='left_only',

<ipython-input-8-fa5bc59f16aa>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmerge["Combined"] = dfmerge["Pilot?"].str.cat(dfmerge[["Principal ID", "Datadate", 'Template Type', 'Statement Type', 'FAW ID']].astype(str), sep="_")


In [10]:
dfmerge.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27337 entries, 69 to 8788
Data columns (total 261 columns):
 #   Column                                                                  Non-Null Count  Dtype         
---  ------                                                                  --------------  -----         
 0   Account ID                                                              27281 non-null  float64       
 1   Account Name                                                            27281 non-null  object        
 2   Principal ID                                                            27337 non-null  int64         
 3   Principal Name                                                          27337 non-null  object        
 4   Principal that Represents the Account                                   27281 non-null  float64       
 5   Account Office                                                          27281 non-null  object        
 6   Datadate             

## Map in Several Mapping Table Values

In [9]:
dfmerge['ISO Country']=dfmerge['Country'].map({'Argentina':'ARG',
'Australia':'AUS',
'Austria':'AUT',
'Bahamas':'BHS',
'Belgium':'BEL',
'Bermuda':'BMU',
'Brazil':'BRA',
'British Virgin Islands':'VGB',
'Canada':'CAN',
'Chile':'CHL',
'China':'CHN',
'Colombia':'COL',
'Costa Rica':'CRI',
'Denmark':'DNK',
'Finland':'FIN',
'France':'FRA',
'Germany':'DEU',
'Greece':'GRC',
'Guatemala':'GTM',
'Honduras':'HND',
'Hong Kong':'HKG',
'Ireland':'IRL',
'Israel':'ISR',
'Italy':'ITA',
'Japan':'JPN',
'Korea':'KOR',
'Korea, South':'KOR',
'Luxembourg':'LUX',
'Madagascar':'MDG',
'Mexico':'MEX',
'Netherlands':'NLD',
'New Zealand':'NZL',
'Norway':'NOR',
'Panama':'PAN',
'Papua New Guinea':'PNG',
'Peru': 'PER',
'Poland': 'POL',
'Portugal':'PRT',
'Russia':'RUS',
'Singapore':'SGP',
'Slovenia':'SVN',
'South Africa':'ZAF',
'South Korea':'KRW',
'Spain':'ESP',
'Sweden':'SWE',
'Switzerland':'CHE',
'Taiwan':'TWN',
'Turkey':'TUR',
'United Arab Emirates':'ARE',
'United Kingdom':'GBR',
'United States':'USA',
'Uruguay':'URY',
'Qatar':'QAR',
'Saudi Arabia':'SAU',
'Dominican Republic':'DOM',
'Isle of Man':'IMN',
'Ecuador':'ECU',
'Malaysia':'MYS',
'Cayman Islands':'CYM',
'United States Virgin Islands':'USA'})

<ipython-input-9-b926a70bc81c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmerge['ISO Country']=dfmerge['Country'].map({'Argentina':'ARG',


In [10]:
dfmerge = dfmerge[dfmerge.columns.drop(list(dfmerge.filter(regex='_y')))]
result_df=dfmerge.drop_duplicates(subset=['Combined'], keep='first')

In [11]:
## ERM Custom Fields
#map in the secondary table that references key lookup dimensions for identifying the proper scoring model
#df_=df18.copy()
#mappingtable2=pd.read_csv(mapping_root+'/mappingtable2.csv')

#Principal ID + Previous Year + As Stated or As Adjusted #need to 
#df18.reset_index()
result_df['principal+previousyear+as']=result_df['Principal ID'].astype(str)+ result_df['Prior Year'].astype(str)+result_df['Pilot?']
result_df['principal+year+state']=result_df['Principal ID'].astype(str)+result_df['Statement Date - Year'].astype(str)+result_df['Pilot?']

#max total asset value in USD
result_df['Max Total Asset Value in USD']=result_df.groupby('Principal ID')['Assets'].transform('max')

#Analyzed Country
result_df['Analyzed Country']=result_df['ISO Country']

#Analyzed Currency
#mappingtableb=pd.read_csv(project_root+'/mappingtableb.csv')
#df6=df.merge(mappingtable2, on='Currency', how= 'left')
result_df['Analyzed Currency']=result_df['ISO Country']

#Asset Category
conditions=[
    (result_df['Max Total Asset Value in USD']<55000000),
    (result_df['Max Total Asset Value in USD']>1000000000),
    (result_df['Max Total Asset Value in USD']<1000000000)]
values=['Below $55M', '> $55M', '> $100M']
result_df['Asset Category']=np.select(conditions, values)

#Country + Asset Category
result_df['Country & Asset Category']=result_df['Analyzed Country']+result_df['Asset Category']

#df19.drop_duplicates(subset = ['Principal ID', 'Account ID'], inplace=True)
#df_a=result_df.copy()
#df__=df_a.merge(mappingtable2, left_on=['Analyzed Country','Asset Category','Country & Asset Category'],right_on=['ISO Country', 'Asset Category','Country & Asset Category'], how='left')
#df['Model']=df__['Risk Calc Model 2.0']

<ipython-input-11-3fa5216d0646>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['principal+previousyear+as']=result_df['Principal ID'].astype(str)+ result_df['Prior Year'].astype(str)+result_df['Pilot?']
<ipython-input-11-3fa5216d0646>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['principal+year+state']=result_df['Principal ID'].astype(str)+result_df['Statement Date - Year'].astype(str)+result_df['Pilot?']
<ipython-input-11-3fa5216d0646>:12: SettingWithCopyWarning: 
A value is tryin

In [12]:
result_df['Risk Calc Model 2.0']=result_df['Country & Asset Category'].map({'ARG> $100M':'Emerging Markets',
'ARG> $55M':'Emerging Markets',
'ARGBelow $55M':'Emerging Markets',
'AUS> $100M':'Australia',
'AUS> $55M':'Australia',
'AUSBelow $55M':'Australia',
'AUT> $100M':'Europe Large Firm',
'AUT> $55M':'Europe Large Firm',
'AUTBelow $55M':'Austria',
'BHS> $100M':'Emerging Markets',
'BHS> $55M':'Emerging Markets',
'BHSBelow $55M':'Emerging Markets',
'BEL> $100M':'Europe Large Firm',
'BEL> $55M':'Europe Large Firm',
'BELBelow $55M':'Europe Large Firm',
'BMU> $100M':'Emerging Markets',
'BMU> $55M':'Emerging Markets',
'BMUBelow $55M':'Emerging Markets',
'BRA> $100M':'Emerging Markets',
'BRA> $55M':'Emerging Markets',
'BRABelow $55M':'Emerging Markets',
'VGB> $100M':'Emerging Markets',
'VGB> $55M':'Emerging Markets',
'VGBBelow $55M':'Emerging Markets',
'CAN> $100M':'North America Large Firm',
'CAN> $55M':'Canada',
'CANBelow $55M':'Canada',
'CHL> $100M':'Emerging Markets',
'CHL> $55M':'Emerging Markets',
'CHLBelow $55M':'Emerging Markets',
'CHN> $100M':'China', 
'CHN> $55M':'China',
'CHNBelow $55M':'China',
'COL> $100M':'Emerging Markets',
'COL> $55M':'Emerging Markets',
'COLBelow $55M':'Emerging Markets',
'CRI> $100M':'Emerging Markets',
'CRI> $55M':'Emerging Markets',
'CRIBelow $55M':'Emerging Markets',
'DNK> $100M':'Europe Large Firm',
'DNK> $55M':'Europe Large Firm',
'DNKBelow $55M':'Denmark',
'FIN> $100M':'Europe Large Firm',
'FIN> $55M':'Europe Large Firm',
'FINBelow $55M':'Finland',
'FRA> $100M':'Europe Large Firm',
'FRA> $55M':'Europe Large Firm',
'FRABelow $55M':'France',
'DEU> $100M':'Europe Large Firm',
'DEU> $55M':'Europe Large Firm',
'DEUBelow $55M':'Germany',
'GRC> $100M':'Europe Large Firm',
'GRC> $55M':'Europe Large Firm',
'GRC> $55M':'Europe Large Firm',
'GTM> $100M':'Emerging Markets',
'GTM> $55M':'Emerging Markets',
'GTMBelow $55M':'Emerging Markets',
'HND> $100M':'Emerging Markets',
'HND> $55M':'Emerging Markets',
'HNDBelow $55M':'Emerging Markets',
'HKG> $100M':'China',
'HKG> $55M':'China',
'HKGBelow $55M':'China',
'IRL> $100M':'Europe Large Firm',
'IRL> $55M':'Europe Large Firm',
'IRLBelow $55M':'Europe Large Firm',
'ISR> $100M':'Emerging Markets',
'ISR> $55M':'Emerging Markets',
'ISRBelow $55M':'Emerging Markets',
'ITA> $100M':'Europe Large Firm',
'ITA> $55M':'Europe Large Firm',
'ITABelow $55M':'Italy',
'JPN> $100M':'Japan',
'JPN> $55M':'Japan',
'JPNBelow $55M':'Japan',
'KOR> $100M':'Korea',
'KOR> $55M':'Korea',
'KORBelow $55M':'Korea',
'KOR> $100M':'Korea',
'KOR> $55M':'Korea',
'KORBelow $55M':'Korea',
'LUX> $100M':'Europe Large Firm',
'LUX> $55M':'Europe Large Firm',
'LUXBelow $55M':'Europe Large Firm',
'MDG> $100M':'Emerging Markets',
'MDG> $55M':'Emerging Markets',
'MDGBelow $55M':'Emerging Markets',
'MEX> $100M':'North America Large Firm',
'MEX> $55M':'Mexico',
'MEXBelow $55M':'Mexico',
'NLD> $100M':'Europe Large Firm',
'NLD> $55M':'Europe Large Firm',
'NLDBelow $55M':'Netherlands',
'NZL> $100M':'Australia',
'NZL> $55M':'Australia',
'NZLBelow $55M':'Australia',
'NOR> $100M':'Europe Large Firm',
'NOR> $55M':'Europe Large Firm',
'NORBelow $55M':'Norway',
'PAN> $100M':'Emerging Markets',
'PAN> $55M':'Emerging Markets',
'PANBelow $55M':'Emerging Markets',
'PNG> $100M':'Emerging Markets',
'PNG> $55M':'Emerging Markets',
'PNGBelow $55M':'Emerging Markets',
'PER> $100M':'Emerging Markets',
'PER> $55M':'Emerging Markets',
'PERBelow $55M':'Emerging Markets',
'POL> $100M':'Europe Large Firm',
'POL> $55M':'Europe Large Firm',
'POLBelow $55M':'Europe Large Firm',
'PRT> $100M':'Europe Large Firm',
'PRT> $55M':'Europe Large Firm',
'PRTBelow $55M':'Portugal',
'RUS> $100M':'Russia',
'RUS> $55M':'Russia',
'RUSBelow $55M':'Russia',
'SGP> $100M':'China',
'SGP> $55M':'China',
'SGPBelow $55M':'China',
'SVN> $100M':'Emerging Markets',
'SVN> $55M':'Emerging Markets',
'SVNBelow $55M':'Emerging Markets',
'ZAF> $100M':'South Africa',
'ZAF> $55M':'South Africa',
'ZAFBelow $55M':'South Africa',
'KRW> $100M':'Korea',
'KRW> $55M':'Korea',
'KRWBelow $55M':'Korea',
'ESP> $100M':'Europe Large Firm',
'ESP> $55M':'Europe Large Firm',
'ESPBelow $55M':'Spain',
'SWE> $100M':'Europe Large Firm',
'SWE> $55M':'Europe Large Firm',
'SWEBelow $55M':'Sweden',
'CHE> $100M':'Europe Large Firm',
'CHE> $55M':'Europe Large Firm',
'CHEBelow $55M':'Switzerland',
'TWN> $100M':'China',
'TWN> $55M':'China',
'TWNBelow $55M':'China',
'TUR> $100M':'Europe Large Firm',
'TUR> $55M':'Europe Large Firm',
'TURBelow $55M':'Europe Large Firm',
'ARE> $100M':'Emerging Markets',
'ARE> $55M':'Emerging Markets',
'AREBelow $55M':'Emerging Markets',
'GBR> $100M':'Europe Large Firm',
'GBR> $55M':'Europe Large Firm',
'GBRBelow $55M':'United Kingdom',
'USA> $100M':'North America Large Firm',
'USA> $55M':'United States',
'USABelow $55M':'United States',
'URY> $100M':'Emerging Markets',
'URY> $55M':'Emerging Markets',
'URYBelow $55M':'Emerging Markets',
'QAR> $100M':'Emerging Markets',
'QAR> $55M':'Emerging Markets',
'QARBelow $55M':'Emerging Markets',
'SAU> $100M':'Emerging Markets',
'SAU> $55M':'Emerging Markets',
'SAUBelow $55M':'Emerging Markets',
'DOM> $100M':'Emerging Markets',
'DOM> $55M':'Emerging Markets',
'DOMBelow $55M':'Emerging Markets',
'IMN> $100M':'Europe Large Firm',
'IMN> $55M':'Europe Large Firm',
'IMNBelow $55M':'Europe Large Firm',
'ECU> $100M':'Emerging Markets',
'ECU> $55M':'Emerging Markets',
'ECUBelow $55M':'Emerging Markets',
'MYS> $100M':'Emerging Markets',
'MYS> $55M':'Emerging Markets',
'MYSBelow $55M':'Emerging Markets',
'CYM> $100M':'Emerging Markets',
'CYM> $55M':'Emerging Markets',
'CYMBelow $55M':'Emerging Markets',
'USA> $100M':'North America Large Firm',
'USA> $55M':'United States',
'USABelow $55M':'United States'})

<ipython-input-12-4c12c2795881>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Risk Calc Model 2.0']=result_df['Country & Asset Category'].map({'ARG> $100M':'Emerging Markets',


In [13]:
result_df[result_df['Risk Calc Model 2.0'].str.isnumeric().fillna(False)]


,Account ID,Account Name,Principal ID,Principal Name,Principal that Represents the Account,Account Office,Datadate,Country,Currency,Units,Statement Period,Statement Type,Template Type,FAW ID,Statement Date - Year,Statement Date - Month,Current Date - Year,Current Date - Month,Industry Classification,Industry Definition,User-Defined Time Period (optional),Financial Statement Only EDF,Assets,Current Assets,Pilot?,Cash,Accounts Receivable,Inventory,Goodwill,Fixed Assets,Liabilities,Current Liabilities,Accounts Payable,Long Term Liabilities,Long Term Debt and Notes Payable,Other Provisions,Shareholders' Equity,Retained Earnings,Sales,Cost of Sales,Gross Profit Margin,"Earnings Before Interest, Taxes, Depreciation and Amortization",Earnings Before Interest and Taxes (EBIT),Interest Income,Interest Expense (-),Net Income (Loss),Depreciation and Amortization - ADJUSTMENTSTORECONCILENETINCOME,Intangibles,Marketable Securities,Long Term Debt Due Within One Year,Other Short Term Interest Bearing Debt,Cash Flow from Operating Activities,Net Income (Loss)1,Capital Expenditures,Free Cash Flow,Fixed Charge Coverage,Interest Bearing Debt,EBITDA,Return on Capital (%),Return on Equity (%),Interest Bearing Debt to Shareholders' Equity,Return on Assets (%),Gross Profit (Loss) to Sales (%),G&A Expense to Sales (%),Net Profit (Loss) to Sales (%),EBITDA to Sales,Interest Bearing Debt to EBITDA,EBITDA to Interest Expense,Interest Bearing Debt to Operating Cash Flow,Cash Flow to Interest Expense,Free Cash Flow to Interest Expense,"Selling, General and Administrative Expenses",Depreciation and Amortization,Other Income (Expense) and Interest,Cash Flow from Investing Activities,Cash Flow from Financing Activities,Proceeds from Issuance of Debt,Issuance of Common Shares,Purchase of Convertible Preferred Securities,Purchase of Common Shares,Payments on Debt,Payments on Capital Lease Obligations,Dividend Payments,Consolidated Id,Product Segment,ProbableGrossExposureAmt,CDs and Marketable Securities,Accounts Receivable - Trade,Accounts Receivable - Other,Accounts and Notes Receivable - Related Parties,Accounts and Notes Receivable - Stockholders',Accounts and Notes Receivable - Officers and Employees,Retainage,Cost and Estimated Earnings in Excess of Billings,Accounts Receivable - Trade.1,Accounts Receivable - Over 90 Days,Retainage.1,Accounts Receivable - Other.1,Accounts and Notes Receivable - Related Parties.1,Accounts and Notes Receivable - Stockholders'.1,Accounts and Notes Receivable - Officers and Employees.1,Other Intangible Assets,Equipment,Buildings,Other Fixed Assets,Real Estate,Working Capital Line,Revolving Line of Credit,Bank Overdraft,Notes Payable - Other,Current Portion - Equipment Debt,Accounts Payable - Trade,Retainage.2,Billings in Excess of Cost and Earned Profit,Accounts Payable - Related Parties,Loans and Payables - Stockholders',Accounts Payable - Other,Revolving Line of Credit.1,Accounts and Notes Payable - Related Parties,Equipment Debt,Accounts and Notes Payable - Stockholders',Notes Payable - Other.1,Real Estate Debt,Other Interest Bearing Debt,Other Long-Term Debt (Non-Interest Bearing),Revenues,Cost of Revenues,Gross Profit,Operating Income,Interest Expense,Net Income Before Taxes,Net Income After Taxes,"Net Income After Taxes, Distributions, and Dividends",Cash flow from Operations,Max Backlog,WC % Case,NW % Case,Interest Coverage,Current Ratio,Quick Ratio,True Cash,Underbillings to Shareholders' Equity (%),Underbillings to Working Capital (%),Liquidity to Working Capital (%),Liquidity to Overbillings (%),Accounts Receivable - Turnover,Accounts Payable - Turnover,Gross Profit (%),G&A / Revenues (%),Net Profit (%),ROE,Debt/Worth,Overhead to Shareholders' Equity (%),Operating Profit (%),G&A Expenses,Dividends and Distributions,Dividends,Subchapter - S Distributions,Investments,Financing,Capital Lease Obligations - Short Term,Operating Lease Obligations - Short Term,Current Portion - Other,Capital Lease Obligations - Long Term,O

In [76]:
result_df['Risk Calc Model 2.0'].unique()

array(['United States', 'Europe Large Firm', 'North America Large Firm',
       'Emerging Markets', nan, 'Canada', 'Australia', 'South Africa',
       'United Kingdom', 'China', 'Germany', 'Spain', 'Mexico', 'Korea',
       'France', 'Japan', 'Netherlands', 'Austria', 'Italy'], dtype=object)

In [13]:
#df__=df__.dropna(how='all', axis=1)
#df__=df__.drop_duplicates(subset=['Account ID', 'Principal ID', 'FAW ID'])
result_df.info(verbose=True, null_counts=True)
df__=result_df.copy()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27215 entries, 69 to 8788
Data columns (total 270 columns):
 #   Column                                                                  Non-Null Count  Dtype         
---  ------                                                                  --------------  -----         
 0   Account ID                                                              27159 non-null  float64       
 1   Account Name                                                            27159 non-null  object        
 2   Principal ID                                                            27215 non-null  int64         
 3   Principal Name                                                          27215 non-null  object        
 4   Principal that Represents the Account                                   27159 non-null  float64       
 5   Account Office                                                          27159 non-null  object        
 6   Datadate             

In [16]:
#Unique value-this combines the Principal ID, Datadate, Template Type/Surety Type, Statement Type-Adjusted or Stated and FAW
df50=pd.DataFrame()
df__["Combined"] = df__["Pilot?"].str.cat(df__[["Principal ID", "Datadate", 'Template Type', 'Statement Type', 'FAW ID']].astype(str), sep="_")
#df['Combined']=df50['Combined']
#combined1=pd.Series(df["Combined"])
#combined1.nunique()

In [17]:
df__.drop_duplicates("Combined")
#df__.combine_first(df19)
df__.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27215 entries, 69 to 8788
Data columns (total 270 columns):
 #   Column                                                                  Non-Null Count  Dtype         
---  ------                                                                  --------------  -----         
 0   Account ID                                                              27159 non-null  float64       
 1   Account Name                                                            27159 non-null  object        
 2   Principal ID                                                            27215 non-null  int64         
 3   Principal Name                                                          27215 non-null  object        
 4   Principal that Represents the Account                                   27159 non-null  float64       
 5   Account Office                                                          27159 non-null  object        
 6   Datadate             

## Calculate New Fields Based-Blend 3 Product Types together

In [14]:
#First we must create a set of data that is sorteed by the Template Type
#create mapping into the cop that is indexed by Risk Calc 2.0
df__['Obligor Name']=result_df['Pilot?']
df__['Obligor Key']=result_df['Principal ID']

#inventory
df__['Inventory']=result_df['Inventory']+result_df['Stock (Inventory)']


In [19]:
df__.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27215 entries, 69 to 8788
Data columns (total 272 columns):
 #   Column                                                                  Non-Null Count  Dtype         
---  ------                                                                  --------------  -----         
 0   Account ID                                                              27159 non-null  float64       
 1   Account Name                                                            27159 non-null  object        
 2   Principal ID                                                            27215 non-null  int64         
 3   Principal Name                                                          27215 non-null  object        
 4   Principal that Represents the Account                                   27159 non-null  float64       
 5   Account Office                                                          27159 non-null  object        
 6   Datadate             

In [15]:
df__['principal+year+state']=df__['principal+year+state'].combine_first(df__['principal+year+state'])

In [16]:
df__['CashandSecurities']=df__['Cash']+df__['CDs and Marketable Securities']
df__['Total Assets']=df__['Assets']

#current liabilities merged correctly

conditions=[
    (result_df['Template Type']=='Contract'),
    (result_df['Template Type']=='Core Commercial'),
    (result_df['Template Type']=='National International')]
values=[(result_df['Inventory']),(result_df ['Stock (Inventory)']), (result_df['Inventory'])]
df__['Inventory']=np.select(conditions, values)


conditions=[
    (result_df['Template Type']=='Contract'),
    (result_df['Template Type']=='Core Commercial'),
    (result_df['Template Type']=='National International')]
values=[(df__['Revolving Line of Credit']+df__['Equipment Debt']+df__['Notes Payable - Other']
+df__['Real Estate Debt']+df__['Other Interest Bearing Debt']+df__["Other Long-Term Debt (Non-Interest Bearing)"]),(df__['Long Term Debt and Notes Payable']), (df__['Loans 1-5 Years - Unsecured']+df__['Loans 1-5 Years - Secured']
+df__['Loans Over 5 Years - Unsecured']+df__['Loans Over 5 Years - Secured']+df__['Non-Recourse Debt 1-5 Years']
+df__['Non-Recourse Debt - Over 5 Years']
)]
df__['LongTermDebt']=np.select(conditions, values)


#Total Liabilities merged correctly
#Retained Earnings Merged Correctly

#sales
conditions=[
    (result_df['Template Type']=='Contract'),
    (result_df['Template Type']=='Core Commercial'),
    (result_df['Template Type']=='National International')]
values=[(result_df['Revenues']),(df__['Sales']), (df__["Revenue (Turnover)"])]
df__['Sales']=np.select(conditions, values)


#ebita
conditions=[
    (result_df['Template Type']=='Contract'),
    (result_df['Template Type']=='Core Commercial'),
    (result_df['Template Type']=='National International')]
values=[(df__['Net Income Before Taxes']+df__['Interest Expense (-)']+df__['Depreciation and Amortization']),(df__['Issuance of Common Shares']), (df__["EBITDA (EBIT+ Depreciation, Amortization and Impairment)"])]
df__['EDITDA']=np.select(conditions, values)


#total interest
conditions=[
    (result_df['Template Type']=='Contract'),
    (result_df['Template Type']=='Core Commercial'),
    (result_df['Template Type']=='National International')]
values=[(df__['Interest Expense (-)']),(df__["Interest Expense (-)"]), (df__['Gross Interest Expense'])]
df__['Total Interest']=np.select(conditions, values)

#net income
conditions=[
    (result_df['Template Type']=='Contract'),
    (result_df['Template Type']=='Core Commercial'),
    (result_df['Template Type']=='National International')]
values=[(df__['Revenues']),(df__['Depreciation and Amortization']), (df__['Net Profit (Loss) Before Dividends'])]
df__['Net Income']=np.select(conditions, values)


In [17]:
#net sales
conditions=[
    (result_df['Template Type']=='Contract'),
    (result_df['Template Type']=='Core Commercial'),
    (result_df['Template Type']=='National International')]
values=[(df__['Net Income After Taxes']),(df__['Sales']), (df__["Revenue (Turnover)"])]
df__['Net Sales']=np.select(conditions, values)

#gross profit
conditions=[
    (result_df['Template Type']=='Contract'),
    (result_df['Template Type']=='Core Commercial'),
    (result_df['Template Type']=='National International')]
values=[(df__['Gross Profit']),(df__['Gross Profit Margin']), (df__['Gross Profit'])]
df__['Gross Profit']=np.select(conditions, values)

#amortization and depreciation
conditions=[
    (result_df['Template Type']=='Contract'),
    (result_df['Template Type']=='Core Commercial'),
    (result_df['Template Type']=='National International')]
values=[(df__['Depreciation and Amortization']),(df__['Depreciation and Amortization']), (df__['Depreciation, Amortization and Impairment'])]
df__['Amortization and Depreciation']=np.select(conditions, values)

df__['Industry Classification']="Sector"

df__['User-Defined Time Period']=1

#tangible fixed assets
conditions=[
    (result_df['Template Type']=='Contract'),
    (result_df['Template Type']=='Core Commercial'),
    (result_df['Template Type']=='National International')]
values=[(df__['Equipment']+df__['Buildings']+df__["Other Fixed Assets"]+df__['Real Estate']+df__['Liabilities']),(df__['Fixed Assets']), (df__['Property']+df__["Plant/Equipment"])]
df__['Tangible Fixed Assets']=np.select(conditions, values)

#intangible fixed assets
conditions=[
    (result_df['Template Type']=='Contract'),
    (result_df['Template Type']=='Core Commercial'),
    (result_df['Template Type']=='National International')]
values=[(df__['Goodwill']+df__['Other Intangible Assets']),(df__['Goodwill']+df__['Fixed Assets']), (df__['Goodwill']+ df__['Intangibles'])]
df__['Intangible Fixed Assets']=np.select(conditions, values)

#accounts receivable
conditions=[
    (result_df['Template Type']=='Contract'),
    (result_df['Template Type']=='Core Commercial'),
    (result_df['Template Type']=='National International')]
values=[(df__['Accounts Receivable - Trade']+df__['Accounts Receivable - Other']+df__['Accounts and Notes Receivable - Related Parties']
+df__['Accounts and Notes Receivable - Officers and Employees']+df__["Accounts and Notes Receivable - Stockholders'.1"]), (df__['Accounts Receivable']), (df__['Trade Accounts Receivable']+df__['Other Accounts Receivable'])]
df__['Accounts Receivable']=np.select(conditions, values)

#accounts payable
conditions=[
    (result_df['Template Type']=='Contract'),
    (result_df['Template Type']=='Core Commercial'),
    (result_df['Template Type']=='National International')]
values=[(df__['Accounts Payable - Trade']+df__['Accounts Payable - Related Parties']+ df__['Accounts Payable - Other']), (df__['Accounts Payable']), (df__['Trade Accounts Payable']+df__['Other Accounts Payable'])]
df__['Accounts Payable']=np.select(conditions, values)


In [18]:
#Prev Year Inventory
df__['Prev Yr Inventory']=df__.groupby('Statement Date - Year')['Inventory'].shift()
df__['Prev Yr Total Assets']=df__.groupby('Statement Date - Year')['Assets'].shift()
df__['Prev Yr LT Debt']=df__.groupby('Statement Date - Year')['LongTermDebt'].shift()
df__['Total Interest Expense']=df__.groupby('Statement Date - Year')['LongTermDebt'].shift()
df__['Prev Yr Sales']=df__.groupby('Statement Date - Year')['Sales'].shift()
df__['Prev Yr Cash and Securities']=df__.groupby('Statement Date - Year')['CashandSecurities'].shift()
df__['Prev Yr Accounts Receivable']=df__.groupby('Statement Date - Year')['Accounts Receivable'].shift()
df__['Prev Yr Total Assets']=df__.groupby('Statement Date - Year')['Total Assets'].shift()
df__['Prev Yr Accounts Payable']=df__.groupby('Statement Date - Year')['Accounts Payable'].shift()
df__['Prev Yr Net Income']=df__.groupby('Statement Date - Year')['Net Income'].shift()
df__['Prev Yr Total Liabilities']=df__.groupby('Statement Date - Year')['Liabilities'].shift()
df__['Prev Yr Net Sales']=df__.groupby('Statement Date - Year')['Net Sales'].shift()

In [27]:
df__.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27215 entries, 69 to 8788
Data columns (total 288 columns):
 #   Column                                                                  Dtype         
---  ------                                                                  -----         
 0   Account ID                                                              float64       
 1   Account Name                                                            object        
 2   Principal ID                                                            int64         
 3   Principal Name                                                          object        
 4   Principal that Represents the Account                                   float64       
 5   Account Office                                                          object        
 6   Datadate                                                                datetime64[ns]
 7   Country                                                  

In [19]:
dfdivide=df__.copy()
dfdivide['Assets']=(df__['Assets']/1000)
dfdivide['Current Assets']=(df__['Current Assets']/1000)
dfdivide['Cash']=(df__['Cash']/1000)
dfdivide['Accounts Receivable']=(df__['Accounts Receivable']/1000)
dfdivide['Inventory']=(df__['Inventory']/1000)
dfdivide['Goodwill']=(df__['Goodwill']/1000)
dfdivide['Fixed Assets']=(df__['Fixed Assets']/1000)
dfdivide['Liabilities']=(df__['Liabilities']/1000) 
dfdivide['Current Liabilities']=(df__['Current Liabilities']/1000)
dfdivide['Accounts Payable']=(df__['Accounts Payable']/1000)
dfdivide['Long Term Liabilities']=(df__['Long Term Liabilities']/1000)
dfdivide['Long Term Debt and Notes Payable']=(df__['Long Term Debt and Notes Payable']/1000)
dfdivide['Other Provisions']=(df__['Other Provisions']/1000)
dfdivide["Shareholders' Equity"]=(df__["Shareholders' Equity"]/1000)
dfdivide['Retained Earnings']=(df__['Retained Earnings']/1000)
dfdivide['Sales']=(df__['Sales']/1000)
dfdivide['Cost of Sales']=(df__['Cost of Sales']/1000)
dfdivide['Gross Profit Margin']=(df__['Gross Profit Margin']/1000)
dfdivide['Earnings Before Interest, Taxes, Depreciation and Amortization']=(df__['Earnings Before Interest, Taxes, Depreciation and Amortization']/1000)
dfdivide['Earnings Before Interest and Taxes (EBIT)']=(df__['Earnings Before Interest and Taxes (EBIT)']/1000)
dfdivide['Interest Income']=(df__['Interest Income']/1000)
dfdivide['Net Income (Loss)']=(df__['Net Income (Loss)']/1000)
dfdivide['Depreciation and Amortization - ADJUSTMENTSTORECONCILENETINCOME']=(df__['Depreciation and Amortization - ADJUSTMENTSTORECONCILENETINCOME']/1000)
dfdivide['Intangibles']=(df__['Intangibles']/1000)
dfdivide['Marketable Securities']=(df__['Marketable Securities']/1000)
dfdivide['Long Term Debt Due Within One Year']=(df__['Long Term Debt Due Within One Year']/1000)
dfdivide['Other Short Term Interest Bearing Debt']=(df__['Other Short Term Interest Bearing Debt']/1000)
dfdivide['Cash Flow from Operating Activities']=(df__['Cash Flow from Operating Activities']/1000)
dfdivide['Net Income (Loss)1']=(df__['Net Income (Loss)1']/1000)
dfdivide['Capital Expenditures']=(df__['Capital Expenditures']/1000)
dfdivide['Free Cash Flow']=(df__['Free Cash Flow']/1000)
dfdivide['Fixed Charge Coverage']=(df__['Fixed Charge Coverage']/1000)
dfdivide['Interest Bearing Debt']=(df__['Interest Bearing Debt']/1000)
dfdivide['EBITDA']=(df__['EBITDA']/1000)
dfdivide["Return on Capital (%)"]=(df__["Return on Capital (%)"]/1000)
dfdivide["Return on Equity (%)"]=(df__["Return on Equity (%)"]/1000)
dfdivide["Interest Bearing Debt to Shareholders' Equity"]=(df__["Interest Bearing Debt to Shareholders' Equity"]/1000)
dfdivide["Return on Assets (%)"]=(df__["Return on Assets (%)"]/1000)
dfdivide["Gross Profit (Loss) to Sales (%)"]=(df__["Gross Profit (Loss) to Sales (%)"]/1000)
dfdivide["G&A Expense to Sales (%)"]=(df__["G&A Expense to Sales (%)"]/1000)
dfdivide["Net Profit (Loss) to Sales (%)"]=(df__["Net Profit (Loss) to Sales (%)"]/1000)
dfdivide["EBITDA to Sales"]=(df__["EBITDA to Sales"]/1000)
dfdivide["Interest Bearing Debt to EBITDA"]=(df__["Interest Bearing Debt to EBITDA"]/1000)
dfdivide["EBITDA to Interest Expense"]=(df__["EBITDA to Interest Expense"]/1000)
dfdivide["Interest Bearing Debt to Operating Cash Flow"]=(df__["Interest Bearing Debt to Operating Cash Flow"]/1000)
dfdivide["Cash Flow to Interest Expense"]=(df__["Cash Flow to Interest Expense"]/1000)
dfdivide["Free Cash Flow to Interest Expense"]=(df__["Free Cash Flow to Interest Expense"]/1000)
dfdivide["Selling, General and Administrative Expenses"]=(df__["Selling, General and Administrative Expenses"]/1000)
dfdivide["Depreciation and Amortization"]=(df__["Depreciation and Amortization"]/1000)
dfdivide["Other Income (Expense) and Interest"]=(df__["Other Income (Expense) and Interest"]/1000)
dfdivide['Cash Flow from Investing Activities']=(df__['Cash Flow from Investing Activities']/1000)
dfdivide["Cash Flow from Financing Activities"]=(df__["Cash Flow from Financing Activities"]/1000)
dfdivide["Proceeds from Issuance of Debt"]=(df__["Proceeds from Issuance of Debt"]/1000)
dfdivide['Issuance of Common Shares']=(df__['Issuance of Common Shares']/1000)
dfdivide['Purchase of Convertible Preferred Securities']=(df__["Purchase of Convertible Preferred Securities"]/1000)
dfdivide['Purchase of Common Shares']=(df__['Purchase of Common Shares']/1000)
dfdivide['Payments on Debt']=(df__['Payments on Debt']/1000)
dfdivide['Payments on Capital Lease Obligations']=(df__['Payments on Capital Lease Obligations']/1000)
dfdivide['Dividend Payments']=(df__['Dividend Payments']/1000)
dfdivide['ProbableGrossExposureAmt']=(df__['ProbableGrossExposureAmt']/1000)
dfdivide['CDs and Marketable Securities']=(df__['CDs and Marketable Securities']/1000)
dfdivide['Accounts Receivable - Trade']=(df__['Accounts Receivable - Trade']/1000)
dfdivide['Accounts Receivable - Other']=(df__['Accounts Receivable - Other']/1000)
dfdivide['Accounts and Notes Receivable - Related Parties']=(df__['Accounts and Notes Receivable - Related Parties']/1000)
dfdivide["Accounts and Notes Receivable - Stockholders'"]=(df__["Accounts and Notes Receivable - Stockholders'"]/1000)
dfdivide['Accounts and Notes Receivable - Officers and Employees']=(df__['Accounts and Notes Receivable - Officers and Employees']/1000)
dfdivide['Retainage']=(df__['Retainage']/1000)
dfdivide['Cost and Estimated Earnings in Excess of Billings']=(df__['Cost and Estimated Earnings in Excess of Billings']/1000)
dfdivide['Accounts Receivable - Trade.1']=(df__['Accounts Receivable - Trade.1']/1000)
dfdivide['Accounts Receivable - Over 90 Days']=(df__['Accounts Receivable - Over 90 Days']/1000)
dfdivide['Retainage.1']=(df__['Retainage.1']/1000)
dfdivide['Accounts Receivable - Other.1']=(df__['Accounts Receivable - Other.1']/1000)
dfdivide['Accounts and Notes Receivable - Related Parties.1']=(df__['Accounts and Notes Receivable - Related Parties.1']/1000)
dfdivide["Accounts and Notes Receivable - Stockholders'.1"]=(df__["Accounts and Notes Receivable - Stockholders'.1"]/1000)
dfdivide["Accounts and Notes Receivable - Officers and Employees.1" ]=(df__["Accounts and Notes Receivable - Officers and Employees.1" ]/1000)
dfdivide['Other Intangible Assets']=(df__['Other Intangible Assets']/1000)
dfdivide['Equipment']=(df__['Equipment']/1000)
dfdivide['Buildings']=(df__['Buildings']/1000)
dfdivide['Other Fixed Assets']=(df__['Other Fixed Assets']/1000)
dfdivide['Real Estate']=(df__['Real Estate']/1000)
dfdivide['Working Capital Line']=(df__['Working Capital Line']/1000)
dfdivide['Revolving Line of Credit']=(df__['Revolving Line of Credit']/1000)
dfdivide['Bank Overdraft']=(df__['Bank Overdraft']/1000)
dfdivide['Notes Payable - Other']=(df__['Notes Payable - Other']/1000)
dfdivide['Current Portion - Equipment Debt']=(df__['Current Portion - Equipment Debt']/1000)
dfdivide['Accounts Payable - Trade']=(df__['Accounts Payable - Trade']/1000)
dfdivide['Retainage.2']=(df__['Retainage.2']/1000)
dfdivide['Billings in Excess of Cost and Earned Profit']=(df__['Billings in Excess of Cost and Earned Profit']/1000)
dfdivide['Accounts Payable - Related Parties']=(df__['Accounts Payable - Related Parties']/1000)
dfdivide["Loans and Payables - Stockholders'"]=(df__["Loans and Payables - Stockholders'"]/1000)
dfdivide['Accounts Payable - Other']=(df__['Accounts Payable - Other']/1000)
dfdivide['Revolving Line of Credit.1']=(df__['Revolving Line of Credit.1']/1000)
dfdivide['Accounts and Notes Payable - Related Parties']=(df__['Accounts and Notes Payable - Related Parties']/1000)
dfdivide['Equipment Debt']=(df__['Equipment Debt']/1000)
dfdivide["Accounts and Notes Payable - Stockholders"]=(df__["Accounts and Notes Payable - Stockholders'"]/1000)
dfdivide['Notes Payable - Other.1']=(df__['Notes Payable - Other.1']/1000)
dfdivide['Real Estate Debt']=(df__['Real Estate Debt']/1000)
dfdivide['Other Interest Bearing Debt']=(df__['Other Interest Bearing Debt']/1000)
dfdivide["Other Long-Term Debt (Non-Interest Bearing)"]=(df__["Other Long-Term Debt (Non-Interest Bearing)"]/1000)
dfdivide['Revenues']=(df__['Revenues']/1000)
dfdivide['Cost of Revenues']=(df__['Cost of Revenues']/1000)
dfdivide['Gross Profit']=(df__['Gross Profit']/1000)
dfdivide['Operating Income']=(df__['Operating Income']/1000)
dfdivide['Interest Expense']=(df__['Interest Expense']/1000)
dfdivide['Net Income Before Taxes']=(df__['Net Income Before Taxes']/1000)
dfdivide['Net Income After Taxes']=(df__['Net Income After Taxes']/1000)
dfdivide['Net Income After Taxes, Distributions, and Dividends']=(df__['Net Income After Taxes, Distributions, and Dividends']/1000)
dfdivide['Cash flow from Operations']=(df__['Cash flow from Operations']/1000)
dfdivide['Max Backlog']=(df__['Max Backlog']/1000)
dfdivide['WC % Case']=(df__['WC % Case']/1000)
dfdivide['NW % Case']=(df__['NW % Case']/1000)
dfdivide['Interest Coverage']=(df__['Interest Coverage']/1000)
dfdivide['Current Ratio']=(df__['Current Ratio']/1000)
dfdivide['Quick Ratio']=(df__['Quick Ratio']/1000)
dfdivide['True Cash']=(df__['True Cash']/1000)
dfdivide["Underbillings to Shareholders' Equity (%)"]=(df__["Underbillings to Shareholders' Equity (%)"]/1000)
dfdivide["Underbillings to Working Capital (%)"]=(df__["Underbillings to Working Capital (%)"]/1000)
dfdivide["Liquidity to Working Capital (%)"]=(df__["Liquidity to Working Capital (%)"]/1000)
dfdivide["Liquidity to Overbillings (%)"]=(df__["Liquidity to Overbillings (%)"]/1000)
dfdivide['Accounts Receivable - Turnover']=(df__['Accounts Receivable - Turnover']/1000)
dfdivide['Accounts Payable - Turnover']=(df__['Accounts Payable - Turnover']/1000)
dfdivide["Gross Profit (%)"]=(df__["Gross Profit (%)"]/1000)
dfdivide["G&A / Revenues (%)"]=(df__["G&A / Revenues (%)"]/1000)
dfdivide["Net Profit (%)"]=(df__["Net Profit (%)"]/1000)
dfdivide['ROE']=(df__['ROE']/1000)
dfdivide['Debt/Worth']=(df__['Debt/Worth']/1000)
dfdivide["Overhead to Shareholders' Equity (%)"]=(df__["Overhead to Shareholders' Equity (%)"]/1000)


In [20]:
dfdivide["Operating Profit (%)"]=(df__["Operating Profit (%)"]/1000)
dfdivide['G&A Expenses']=(df__['G&A Expenses']/1000)
dfdivide['Dividends and Distributions']=(df__['Dividends and Distributions']/1000)
dfdivide['Dividends']=(df__['Dividends']/1000)
dfdivide['Subchapter - S Distributions']=(df__['Subchapter - S Distributions']/1000)
dfdivide['Investments']=(df__['Investments']/1000)
dfdivide['Financing']=(df__['Financing']/1000)
dfdivide['Capital Lease Obligations - Short Term']=(df__['Capital Lease Obligations - Short Term']/1000)
dfdivide['Operating Lease Obligations - Short Term']=(df__['Operating Lease Obligations - Short Term']/1000)
dfdivide['Current Portion - Other']=(df__['Current Portion - Other']/1000)
dfdivide['Capital Lease Obligations - Long Term']=(df__['Capital Lease Obligations - Long Term']/1000)
dfdivide['Operating Lease Obligations - Long Term']=(df__['Operating Lease Obligations - Long Term']/1000)
dfdivide['Trade Accounts Receivable']=(df__['Trade Accounts Receivable']/1000)
dfdivide['Other Accounts Receivable']=(df__['Other Accounts Receivable']/1000)
dfdivide["Stock (Inventory)"]=(df__["Stock (Inventory)"]/1000)
dfdivide["Work In Progress (WIP)"]=(df__["Work In Progress (WIP)"]/1000)
dfdivide['Advances']=(df__['Advances']/1000)
dfdivide['Property']=(df__['Property']/1000)
dfdivide['Plant/Equipment']=(df__['Plant/Equipment']/1000)
dfdivide['Long Term Receivables']=(df__['Long Term Receivables']/1000)
dfdivide['Long Term WIP']=(df__['Long Term WIP']/1000)
dfdivide['Current Bank Debt - Unsecured']=(df__['Current Bank Debt - Unsecured']/1000)
dfdivide['Current Bank Debt - Secured']=(df__['Current Bank Debt - Secured']/1000)
dfdivide['Current Portion of Long Term Debt - Unsecured']=(df__['Current Portion of Long Term Debt - Unsecured']/1000)
dfdivide['Current Portion of Long Term Debt - Secured']=(df__['Current Portion of Long Term Debt - Secured']/1000)
dfdivide['Current Portion of Non-Recourse Debt']=(df__['Current Portion of Non-Recourse Debt']/1000)
dfdivide['Trade Accounts Payable']=(df__['Trade Accounts Payable']/1000)
dfdivide['Other Accounts Payable']=(df__['Other Accounts Payable']/1000)
dfdivide['Advances from Customers']=(df__['Advances from Customers']/1000)
dfdivide['Loans 1-5 Years - Unsecured']=(df__['Loans 1-5 Years - Unsecured']/1000)
dfdivide['Loans 1-5 Years - Secured']=(df__['Loans 1-5 Years - Secured']/1000)
dfdivide['Loans Over 5 Years - Unsecured']=(df__['Loans Over 5 Years - Unsecured']/1000)
dfdivide['Loans Over 5 Years - Secured']=(df__['Loans Over 5 Years - Secured']/1000)
dfdivide['Non-Recourse Debt 1-5 Years']=(df__['Non-Recourse Debt 1-5 Years']/1000)
dfdivide['Non-Recourse Debt - Over 5 Years']=(df__['Non-Recourse Debt - Over 5 Years']/1000)
dfdivide['Long Term Accounts Payable']=(df__['Long Term Accounts Payable']/1000)
dfdivide['Advances from Customers.1']=(df__['Advances from Customers.1']/1000)
dfdivide['Provisions, Other']=(df__['Provisions, Other']/1000)
dfdivide["Stockholders' Equity"]=(df__["Stockholders' Equity"]/1000)
dfdivide["Revenue (Turnover)"]=(df__["Revenue (Turnover)"]/1000)
dfdivide['Prev Yr Inventory']=(df__['Prev Yr Inventory']/1000)
dfdivide['Prev Yr Total Assets']=(df__['Prev Yr Total Assets']/1000)
dfdivide['Prev Yr Accounts Payable']=(df__['Prev Yr Accounts Payable']/1000)
dfdivide['Prev Yr LT Debt']=(df__['Prev Yr LT Debt']/1000)
dfdivide['Total Liabilities']=(df__['Liabilities']/1000)
dfdivide['Prev Yr Sales']=(df__['Prev Yr Sales']/1000)
dfdivide['Prev Yr Net Income']=(df__['Prev Yr Net Income']/1000)


In [21]:
dfdivide['obligorName']=dfdivide['Obligor Name']
dfdivide['entityIdentifier']=dfdivide['Obligor Key']
dfdivide['financialStatementDate']=dfdivide['Datadate']
dfdivide['cash']=dfdivide['Cash']
dfdivide['totalAssets']=dfdivide['Total Assets']
dfdivide['totalAccountsPayable']=dfdivide['Accounts Payable']
dfdivide['totalCurrentLiabilities']=dfdivide['Current Liabilities']              
dfdivide['totalLiabilitiesPreviousYear']=dfdivide['Prev Yr Total Liabilities']
dfdivide['totalSales']=dfdivide['Sales']
dfdivide['totalSalesPreviousYear']=dfdivide['Prev Yr Sales']
dfdivide['totalOperatingProfit']=dfdivide['Operating Income']
dfdivide['totalAmortizationAndDepreciation']=dfdivide['Depreciation, Amortization and Impairment']
dfdivide['Interest Expense']=dfdivide['Net Income']
dfdivide['primaryIndustryClassification']=dfdivide['Industry Classification']
dfdivide['netIncome']=dfdivide['Net Income']
dfdivide['Industry']=dfdivide['Industry Classification']
dfdivide['primaryIndustry']=dfdivide['Industry']
dfdivide['pdTerm']= dfdivide[ "User-Defined Time Period (optional)"]
dfdivide['isTTC']=dfdivide['Financial Statement Only EDF'] 
dfdivide['RiskCalc Model']=dfdivide['Risk Calc Model 2.0']
dfdivide['Principal ID + Year']=dfdivide['Principal ID']+dfdivide['Statement Date - Year']
dfdivide ['Cash and Marketable Securities']=dfdivide['Cash']+dfdivide['CDs and Marketable Securities']
dfdivide['Cash and Cash Equivalents']=dfdivide['Cash and Cash Equivalents at Beginning of Year']+dfdivide['Increase (Decrease) in Cash and Cash Equivalent']
dfdivide["Exchange Rate"]=" "

In [64]:
dfdivide.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27215 entries, 69 to 8788
Data columns (total 316 columns):
 #   Column                                                                  Non-Null Count  Dtype         
---  ------                                                                  --------------  -----         
 0   Account ID                                                              27159 non-null  float64       
 1   Account Name                                                            27159 non-null  object        
 2   Principal ID                                                            27215 non-null  int64         
 3   Principal Name                                                          27215 non-null  object        
 4   Principal that Represents the Account                                   27159 non-null  float64       
 5   Account Office                                                          27159 non-null  object        
 6   Datadate             

In [116]:
dfdivide.head(30)

,Account ID,Account Name,Principal ID,Principal Name,Principal that Represents the Account,Account Office,Datadate,Country,Currency,Units,Statement Period,Statement Type,Template Type,FAW ID,Statement Date - Year,Statement Date - Month,Current Date - Year,Current Date - Month,Industry Classification,Industry Definition,User-Defined Time Period (optional),Financial Statement Only EDF,Assets,Current Assets,Pilot?,Cash,Accounts Receivable,Inventory,Goodwill,Fixed Assets,Liabilities,Current Liabilities,Accounts Payable,Long Term Liabilities,Long Term Debt and Notes Payable,Other Provisions,Shareholders' Equity,Retained Earnings,Sales,Cost of Sales,Gross Profit Margin,"Earnings Before Interest, Taxes, Depreciation and Amortization",Earnings Before Interest and Taxes (EBIT),Interest Income,Interest Expense (-),Net Income (Loss),Depreciation and Amortization - ADJUSTMENTSTORECONCILENETINCOME,Intangibles,Marketable Securities,Long Term Debt Due Within One Year,Other Short Term Interest Bearing Debt,Cash Flow from Operating Activities,Net Income (Loss)1,Capital Expenditures,Free Cash Flow,Fixed Charge Coverage,Interest Bearing Debt,EBITDA,Return on Capital (%),Return on Equity (%),Interest Bearing Debt to Shareholders' Equity,Return on Assets (%),Gross Profit (Loss) to Sales (%),G&A Expense to Sales (%),Net Profit (Loss) to Sales (%),EBITDA to Sales,Interest Bearing Debt to EBITDA,EBITDA to Interest Expense,Interest Bearing Debt to Operating Cash Flow,Cash Flow to Interest Expense,Free Cash Flow to Interest Expense,"Selling, General and Administrative Expenses",Depreciation and Amortization,Other Income (Expense) and Interest,Cash Flow from Investing Activities,Cash Flow from Financing Activities,Proceeds from Issuance of Debt,Issuance of Common Shares,Purchase of Convertible Preferred Securities,Purchase of Common Shares,Payments on Debt,Payments on Capital Lease Obligations,Dividend Payments,Consolidated Id,Product Segment,ProbableGrossExposureAmt,CDs and Marketable Securities,Accounts Receivable - Trade,Accounts Receivable - Other,Accounts and Notes Receivable - Related Parties,Accounts and Notes Receivable - Stockholders',Accounts and Notes Receivable - Officers and Employees,Retainage,Cost and Estimated Earnings in Excess of Billings,Accounts Receivable - Trade.1,Accounts Receivable - Over 90 Days,Retainage.1,Accounts Receivable - Other.1,Accounts and Notes Receivable - Related Parties.1,Accounts and Notes Receivable - Stockholders'.1,Accounts and Notes Receivable - Officers and Employees.1,Other Intangible Assets,Equipment,Buildings,Other Fixed Assets,Real Estate,Working Capital Line,Revolving Line of Credit,Bank Overdraft,Notes Payable - Other,Current Portion - Equipment Debt,Accounts Payable - Trade,Retainage.2,Billings in Excess of Cost and Earned Profit,Accounts Payable - Related Parties,Loans and Payables - Stockholders',Accounts Payable - Other,Revolving Line of Credit.1,Accounts and Notes Payable - Related Parties,Equipment Debt,Accounts and Notes Payable - Stockholders',Notes Payable - Other.1,Real Estate Debt,Other Interest Bearing Debt,Other Long-Term Debt (Non-Interest Bearing),Revenues,Cost of Revenues,Gross Profit,Operating Income,Interest Expense,Net Income Before Taxes,Net Income After Taxes,"Net Income After Taxes, Distributions, and Dividends",Cash flow from Operations,Max Backlog,WC % Case,NW % Case,Interest Coverage,Current Ratio,Quick Ratio,True Cash,Underbillings to Shareholders' Equity (%),Underbillings to Working Capital (%),Liquidity to Working Capital (%),Liquidity to Overbillings (%),Accounts Receivable - Turnover,Accounts Payable - Turnover,Gross Profit (%),G&A / Revenues (%),Net Profit (%),ROE,Debt/Worth,Overhead to Shareholders' Equity (%),Operating Profit (%),G&A Expenses,Dividends and Distributions,Dividends,Subchapter - S Distributions,Investments,Financing,Capital Lease Obligations - Short Term,Operating Lease Obligations - Short Term,Current Portion - Other,Capital Lease Obligations - Long Term,O

In [154]:
Canada11=pd.DataFrame()
    
Canada11=dfdivide[['Obligor Name', 'Obligor Key', 'Statement Date - Year', 'Statement Date - Month', 'Current Date - Year' 
                     , 'Current Date - Month', 'Inventory', 'Prev Yr Inventory', 'Accounts Receivable', 'Prev Yr Accounts Receivable',
                     'CashandSecurities', 'Total Assets', 'Prev Yr Total Assets', 'Accounts Payable', 'Prev Yr Accounts Payable',
                      'Current Liabilities','LongTermDebt', 'Total Liabilities', 'Retained Earnings', 'Sales', 'Prev Yr Sales',
                     "Earnings Before Interest and Taxes (EBIT)", 'Total Interest Expense', 'Prev Yr Net Income', 'Industry Classification','Industry Definition', 'Risk Calc Model 2.0']] 

Canada111 = Canada11[(Canada11['Risk Calc Model 2.0']=="Canada")]
Canada2=Canada111.copy()
Canada3=Canada111.copy()
Canada4=Canada111.copy()
Canada5=Canada111.copy()
Canada=Canada111.append([Canada2,Canada3, Canada4, Canada5], ignore_index=True)


In [155]:
Canada.head()

,Obligor Name,Obligor Key,Statement Date - Year,Statement Date - Month,Current Date - Year,Current Date - Month,Inventory,Prev Yr Inventory,Accounts Receivable,Prev Yr Accounts Receivable,CashandSecurities,Total Assets,Prev Yr Total Assets,Accounts Payable,Prev Yr Accounts Payable,Current Liabilities,LongTermDebt,Total Liabilities,Retained Earnings,Sales,Prev Yr Sales,Earnings Before Interest and Taxes (EBIT),Total Interest Expense,Prev Yr Net Income,Industry Classification,Industry Definition,Risk Calc Model 2.0
0,Stated,966407512,2019,12,2022,06,0.0,0.0,0.0,0.0,NaN,4.679900e+10,3.303967e+06,0.0,0.0,9129000.0,0.0,23930000.0,7101000.0,0.0,0.0,2308000.0,0.0,0.0,Sector,Agriculture (Commercial Surety),Canada
1,Stated,966407512,2017,12,2022,06,0.0,0.0,0.0,0.0,NaN,1.699800e+10,3.197300e+06,0.0,0.0,1602000.0,0.0,8695000.0,6242000.0,0.0,0.0,461000.0,0.0,0.0,Sector,Agriculture (Commercial Surety),Canada
2,Stated,966407512,2018,12,2022,06,0.0,0.0,0.0,0.0,NaN,4.550200e+10,1.105259e+07,0.0,0.0,8335000.0,0.0,21077000.0,7745000.0,0.0,0.0,707000.0,0.0,0.0,Sector,Agriculture (Commercial Surety),Canada
3,Stated,966407512,2015,12,2022,06,0.0,0.0,0.0,0.0,NaN,1.746900e+10,2.440493e+05,0.0,0.0,1747000.0,0.0,9087000.0,6455000.0,0.0,0.0,2030000.0,0.0,0.0,Sector,Agriculture (Commercial Surety),Canada
4,Stated,966407512,2016,12,2022,06,0.0,0.0,0.0,0.0,NaN,1.725500e+10,7.407000e+06,0.0,0.0,1697000.0,0.0,9056000.0,6204000.0,0.0,0.0,618000.0,0.0,0.0,Sector,Agriculture (Commercial Surety),Canada


In [176]:
EmergingMarkets=pd.DataFrame()

EmergingMarkets=dfdivide[['Obligor Name', 'Obligor Key', 'Statement Date - Year', 'Statement Date - Month', 'Current Date - Year' 
                     , 'Current Date - Month', 'Inventory', 'Current Assets','Total Assets','Current Liabilities','Total Liabilities',
                     'Sales', 'Prev Yr Sales', 'Accounts Payable', 'Prev Yr Accounts Payable',
                       'Retained Earnings', 'Sales', 'Prev Yr Sales', 'Amortization and Depreciation', "Operating Profit (%)",
                         'Interest Expense', 'Net Income', 'Country', 'Industry Classification', 'Industry Definition',
                     'Financial Statement Only EDF','Risk Calc Model 2.0']]
EmergingMarkets1=EmergingMarkets[(EmergingMarkets['Risk Calc Model 2.0']=="Emerging Markets")]
EmergingMarkets1.drop(['Risk Calc Model 2.0'],axis=1)
Emerging2=EmergingMarkets1.copy()
Emerging3=EmergingMarkets1.copy()
Emerging4=EmergingMarkets1.copy()
Emerging5=EmergingMarkets1.copy()
Emerging12=EmergingMarkets1.append(Emerging2, ignore_index=True)
Emerging123=Emerging12.append(Emerging3,ignore_index=True)
Emerging1234=Emerging123.append(Emerging4,ignore_index=True)
Emerging=Emerging1234.append(Emerging5,ignore_index=True)

In [177]:
Emerging.head()

,Obligor Name,Obligor Key,Statement Date - Year,Statement Date - Month,Current Date - Year,Current Date - Month,Inventory,Current Assets,Total Assets,Current Liabilities,Total Liabilities,Sales,Prev Yr Sales,Accounts Payable,Prev Yr Accounts Payable,Retained Earnings,Sales,Prev Yr Sales,Amortization and Depreciation,Operating Profit (%),Interest Expense,Net Income,Country,Industry Classification,Industry Definition,Financial Statement Only EDF,Risk Calc Model 2.0
0,Stated,35061421,2016,12,2022,06,0.0,122965000.0,1.597860e+11,86825000.0,111511000.0,0.0,0.0,0.0,0.0,23613000.0,0.0,0.0,0.0,NaN,0.0,0.0,Bermuda,Sector,Core Commercial,NaN,Emerging Markets
1,Stated,35061421,2018,12,2022,06,0.0,129073000.0,1.677710e+11,92011000.0,117459000.0,0.0,0.0,0.0,0.0,31700000.0,0.0,0.0,0.0,NaN,0.0,0.0,Bermuda,Sector,Core Commercial,NaN,Emerging Markets
2,Stated,35061421,2017,12,2022,06,0.0,128614000.0,1.729800e+11,91641000.0,115850000.0,0.0,0.0,0.0,0.0,27474000.0,0.0,0.0,0.0,NaN,0.0,0.0,Bermuda,Sector,Core Commercial,NaN,Emerging Markets
3,Stated,35061421,2019,12,2022,06,0.0,136008000.0,1.769430e+11,94639000.0,121612000.0,0.0,0.0,0.0,0.0,36142000.0,0.0,0.0,0.0,NaN,0.0,0.0,Bermuda,Sector,Core Commercial,NaN,Emerging Markets
4,Stated,976419312,2017,8,2022,06,0.0,12097289.0,2.268989e+10,9824279.0,12979690.0,0.0,0.0,0.0,0.0,7081855.0,0.0,0.0,0.0,NaN,0.0,0.0,Argentina,Sector,Core Commercial,NaN,Emerging Markets


In [191]:
EuropeLargeFirm=pd.DataFrame()

EuropeLargeFirm=dfdivide[['Obligor Name', 'Obligor Key', 'Statement Date - Year', 'Statement Date - Month', 'Current Date - Year' 
                     , 'Current Date - Month', 'Cash and Cash Equivalents', 'Total Assets', 'Prev Yr Total Assets', 'Accounts Payable',
                     'Current Liabilities','Total Liabilities','Sales', 'Prev Yr Sales', 'Amortization and Depreciation',
                    'Gross Profit', 'Interest Expense', 'Net Income', 'Prev Yr Net Income', 'Country', 
                    'Industry Classification', 'Industry Definition', "User-Defined Time Period (optional)",
                     'Financial Statement Only EDF', 'Currency', 'Exchange Rate', 'Risk Calc Model 2.0']]
EuropeLargeFirm1=EuropeLargeFirm[(EuropeLargeFirm['Risk Calc Model 2.0']=="Europe Large Firm")]
EuropeLargeFirm11=EuropeLargeFirm1.drop(['Risk Calc Model 2.0'],axis=1)
Europe2=EuropeLargeFirm11.copy()
Europe3=EuropeLargeFirm11.copy()
Europe4=EuropeLargeFirm11.copy()
Europe5=EuropeLargeFirm11.copy()
Europe12=EuropeLargeFirm11.append(Europe2, ignore_index=True)
Europe123=Europe12.append(Europe3,ignore_index=True)
Europe1234=Europe123.append(Europe4,ignore_index=True)
Europe=Europe1234.append(Europe5,ignore_index=True)

In [197]:
Mexicos=pd.DataFrame()

Mexicos=dfdivide[['Obligor Name', 'Obligor Key', 'Statement Date - Year', 'Statement Date - Month', 'Current Date - Year' 
                     , 'Current Date - Month', 'Cash and Marketable Securities', 'Total Assets', 'Total Liabilities','Net Sales', 
                      'Prev Yr Net Sales', 'Gross Profit', 'Amortization and Depreciation', "Operating Profit (%)", 'Total Interest Expense'
                    ,"User-Defined Time Period (optional)", 'Financial Statement Only EDF', 'Risk Calc Model 2.0']]
Mexicos1=Mexicos[(Mexicos['Risk Calc Model 2.0']=="Mexico")]
Mexicos11=Mexicos1.drop(['Risk Calc Model 2.0'], axis=1)
Mexico2=Mexicos11.copy()
Mexico3=Mexicos11.copy()
Mexico4=Mexicos11.copy()
Mexico5=Mexicos11.copy()
Mexico12=Mexicos11.append(Mexico2, ignore_index=True)
Mexico123=Mexico12.append(Mexico3,ignore_index=True)
Mexico1234=Mexico123.append(Mexico4,ignore_index=True)
Mexico=Mexico1234.append(Mexico5,ignore_index=True)
Mexico=Mexico.set_index(['Obligor Name','Obligor Key'])
NorthAmericamax.sort_values(by=['Statement Date - Year', 'Statement Date - Month'])

In [198]:
Mexico.head()

,Obligor Name,Obligor Key,Statement Date - Year,Statement Date - Month,Current Date - Year,Current Date - Month,Cash and Marketable Securities,Total Assets,Total Liabilities,Net Sales,Prev Yr Net Sales,Gross Profit,Amortization and Depreciation,Operating Profit (%),Total Interest Expense,User-Defined Time Period (optional),Financial Statement Only EDF
0,Stated,519252132,2015,12,2022,06,NaN,3.081907e+09,1335676.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN
1,Adjusted,519252132,2015,12,2022,06,NaN,3.081907e+09,1335676.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN
2,Adjusted,956435212,2018,12,2022,06,NaN,2.633170e+11,178743000.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN
3,Adjusted,956435212,2016,12,2022,06,NaN,2.451650e+11,170089000.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN
4,Adjusted,956435212,2017,12,2022,06,NaN,2.592490e+11,182225000.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN


In [25]:
NorthAmericaLarge=pd.DataFrame()

NorthAmericaLarge=dfdivide[['Obligor Name', 'Obligor Key', 'Statement Date - Year', 'Statement Date - Month', 'Current Date - Year' 
                     , 'Current Date - Month', 'Cash and Marketable Securities', 'Total Assets', 'Total Liabilities','Net Sales', 
                      'Prev Yr Net Sales', 'Gross Profit', 'Amortization and Depreciation', 'Operating Income', 'Total Interest Expense'
                    , "User-Defined Time Period (optional)", 'Financial Statement Only EDF', 'Risk Calc Model 2.0']]
NorthAmericaLarge1=NorthAmericaLarge[NorthAmericaLarge['Risk Calc Model 2.0']=="North America Large Firm"]
NorthAmericaLarge11=NorthAmericaLarge1.drop(['Risk Calc Model 2.0'], axis=1)
NorthAmerica2=NorthAmericaLarge11.copy()
NorthAmerica3=NorthAmericaLarge11.copy()
NorthAmerica4=NorthAmericaLarge11.copy()
NorthAmerica5=NorthAmericaLarge11.copy()
NorthAmerica12=NorthAmericaLarge11.append(NorthAmerica2, ignore_index=True)
NorthAmerica123=NorthAmerica12.append(NorthAmerica3,ignore_index=True)
NorthAmerica1234=NorthAmerica123.append(NorthAmerica4,ignore_index=True)
NorthAmerica=NorthAmerica1234.append(NorthAmerica5,ignore_index=True)
NorthAmericamax=NorthAmerica.set_index(['Obligor Name','Obligor Key'])
NorthAmericamax.sort_values(by=['Statement Date - Year', 'Statement Date - Month'])

Statement Date - Year  Statement Date - Month  \
Obligor Name Obligor Key                                                  
Stated       565634221                     2015                       1   
             986390412                     2015                       1   
             381067191                     2015                       1   
             786480612                     2015                       1   
Adjusted     381067191                     2015                       1   
...                                         ...                     ...   
             786480612                     2022                       1   
Stated       65603221                      2022                       1   
             786480612                     2022                       1   
Adjusted     65603221                      2022                       1   
             786480612                     2022                       1   

                         Current Date - Year Current Date - Month  \
Obligor Name Obligor Key                                            
Stated       565634221                  2022                   07   
             986390412                  2022                   07   
             381067191                  2022                   07   
             786480612                  2022                   07   
Adjusted     381067191                  2022                   07   
...                                      ...                  ...   
             786480612                  2022                   07   
Stated       65603221                   2022                   07   
             786480612                  2022                   07   
Adjusted     65603221                   2022                   07   
             786480612                  2022                   07   

                          Cash and Marketable Securities  Total Assets  \
Obligor Name Obligor Key                                                 
Stated       565634221                               NaN   636929000.0   
             986390412                               NaN   709719000.0   
             381067191                               NaN   383251000.0   
             786480612                               NaN   461481000.0   
Adjusted     381067191                               NaN   383251000.0   
...                                                  ...           ...   
             786480612                               NaN   360964000.0   
Stated       65603221                                NaN   753953000.0   
             786480612                               NaN   360964000.0   
Adjusted     65603221                                NaN   727582000.0   
             786480612                               NaN   360964000.0   

                          Total Liabilities  Net Sales  Prev Yr Net Sales  \
Obligor Name Obligor Key                                                    
Stated       565634221             467297.0        0.0                0.0   
             986390412             450723.0        0.0                0.0   
             381067191             129912.0        0.0                0.0   
             786480612             389503.0        0.0                0.0   
Adjusted     381067191             129912.0        0.0                0.0   
...                                     ...        ...                ...   
             786480612             404457.0        0.0                0.0   
Stated       65603221              486644.0        0.0                0.0   
             786480612             404457.0        0.0                0.0   
Adjusted     65603221              486644.0        0.0                0.0   
             786480612             404457.0        0.0                0.0   

                          Gross Profit  Amortization and Depreciation  \
Obligor Name Obligor Key                                                
Stated       565634221             0.0   

In [205]:
NorthAmerica.head()

,Obligor Name,Obligor Key,Statement Date - Year,Statement Date - Month,Current Date - Year,Current Date - Month,Cash and Marketable Securities,Total Assets,Total Liabilities,Net Sales,Prev Yr Net Sales,Gross Profit,Amortization and Depreciation,Operating Income,Total Interest Expense,User-Defined Time Period (optional),Financial Statement Only EDF
0,Stated,65057721,2016,12,2022,06,NaN,340500000.0,288300.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN
1,Stated,65057721,2017,12,2022,06,NaN,337300000.0,306800.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN
2,Stated,65057721,2018,12,2022,06,NaN,306900000.0,334900.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN
3,Stated,199182232,2015,3,2022,06,NaN,315411000.0,135039.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN
4,Stated,935057021,2016,9,2022,06,NaN,297159000.0,160545.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN


In [237]:
Australias=pd.DataFrame()
Australias['Current Mapped Date']=dfdivide['Statement Date - Month']+dfdivide['Statement Date - Year']
Australias=dfdivide[['obligorName', 'entityIdentifier', 'financialStatementDate', 'cash' 
                     , 'totalAssets','totalAccountsPayable' ,'totalCurrentLiabilities', 'totalLiabilitiesPreviousYear',
                      'totalSales','Sales', 'totalSalesPreviousYear',  'totalOperatingProfit', 'totalAmortizationAndDepreciation',
                     'Interest Expense', 'Net Income','primaryIndustryClassification','Industry',"User-Defined Time Period (optional)", 'isTTC', 'Risk Calc Model 2.0']]
Australia1=Australias[Australias['Risk Calc Model 2.0']=="Australia"]
Australia1.rename(columns={'Obligor Name':'obligorName', 'Obligor Key':'entityIdentifier','Datadate':'financialStatementDate'
                                 ,'Cash' :'cash','Total Assets':'totalAssets', 'Accounts Payable':'totalAccountsPayable' ,
                                       'Current Liabilities': 'totalCurrentLiabilities','Prev Yr Total Liabilities':'totalLiabilitiesPreviousYear'
                                      ,'Sales': 'totalSales','Prev Yr Sales': 'totalSalesPreviousYear', 'Operating Profit':'totalOperatingProfit' 
                                      ,'Depreciation, Amortization and Impairment':'totalAmortizationAndDepreciation', 'Interest Expense':'totalInterestExpense',
                                       'Net Income': 'netIncome','Industry Classification':'primaryIndustryClassification', 'Industry': 'primaryIndustry',
                                      'User Defined Time Period': 'pdTerm', 'Financial Statement Only EDF': 'isTTC'})
Australia11=Australia1.drop(['Risk Calc Model 2.0'], axis=1)
Australia2=Australia11.copy()
Australia3=Australia11.copy()
Australia4=Australia11.copy()
Australia5=Australia11.copy()
Australia12=Australia11.append(Australia2, ignore_index=True)
Australia123=Australia12.append(Australia3,ignore_index=True)
Australia1234=Australia123.append(Australia4,ignore_index=True)
Australia=Australia1234.append(Australia5,ignore_index=True)
Australiamax=Australia.set_index(['entityIdentifier','obligorName'])
Australiamax.sort_values(by='financialStatementDate')


financialStatementDate       cash   totalAssets  \
entityIdentifier obligorName                                                   
966413912        Stated                  2015-03-31    67000.0  2.044500e+09   
                 Stated                  2015-03-31    67000.0  2.044500e+09   
                 Adjusted                2015-03-31    67000.0  1.298400e+09   
                 Adjusted                2015-03-31    67000.0  1.298400e+09   
                 Stated                  2015-03-31    67000.0  2.044500e+09   
...                                             ...        ...           ...   
999867685        Stated                  2021-12-31   171024.0  6.402800e+08   
901905042        Adjusted                2021-12-31   180200.0  2.681400e+09   
999867685        Stated                  2021-12-31   171024.0  6.402800e+08   
122061742        Adjusted                2021-12-31  1939700.0  8.808400e+09   
999867685        Adjusted                2021-12-31   171024.0  6.402800e+08   

                              totalAccountsPayable  totalCurrentLiabilities  \
entityIdentifier obligorName                                                  
966413912        Stated                        0.0                 377900.0   
                 Stated                        0.0                 377900.0   
                 Adjusted                      0.0                 228200.0   
                 Adjusted                      0.0                 228200.0   
                 Stated                        0.0                 377900.0   
...                                            ...                      ...   
999867685        Stated                        0.0                 306196.0   
901905042        Adjusted                      0.0                1137700.0   
999867685        Stated                        0.0                 306196.0   
122061742        Adjusted                      0.0                5071700.0   
999867685        Adjusted                      0.0                 306196.0   

                              totalLiabilitiesPreviousYear  totalSales  Sales  \
entityIdentifier obligorName                                                    
966413912        Stated                       5.206000e+10         0.0    0.0   
                 Stated                       5.206000e+10         0.0    0.0   
                 Adjusted                     2.845090e+10         0.0    0.0   
                 Adjusted                     2.845090e+10         0.0    0.0   
                 Stated                       5.206000e+10         0.0    0.0   
...                                                    ...         ...    ...   
999867685        Stated                       7.407100e+08         0.0    0.0   
901905042        Adjusted                     1.660800e+09         0.0    0.0   
999867685        Stated                       7.407100e+08         0.0    0.0   
122061742        Adjusted                     1.410545e+10         0.0    0.0   
999867685        Adjusted                     7.407100e+08         0.0    0.0   

                              totalSalesPreviousYear  totalOperatingProfit  \
entityIdentifier obligorName                                                 
966413912        Stated                          0.0                   NaN   
                 Stated                          0.0                   NaN   
                 Adjusted                        0.0                   NaN   
                 Adjusted                        0.0                   NaN   
                 Stated                          0.0                   NaN   
...                                              ...                   ...   
999867685        Stated                          0.0                   NaN   
901905042        Adjusted                        0.0                   NaN   
999867685        Stated                          0.0                   NaN   
122061742        Adjusted                       

In [231]:
Australiamax.head()

financialStatementDate      cash   totalAssets  \
entityIdentifier obligorName                                                  
966413912        Stated                  2018-03-31  281600.0  2.351000e+09   
                 Stated                  2019-03-31   78700.0  4.032600e+09   
                 Stated                  2020-03-31  144400.0  4.028300e+09   
                 Stated                  2016-03-31  107100.0  2.040400e+09   
                 Stated                  2021-03-31  208500.0  4.089000e+09   

                              totalAccountsPayable  totalCurrentLiabilities  \
entityIdentifier obligorName                                                  
966413912        Stated                        0.0                 394600.0   
                 Stated                        0.0                 483000.0   
                 Stated                        0.0                 510000.0   
                 Stated                        0.0                 399800.0   
                 Stated                        0.0                 901100.0   

                              totalLiabilitiesPreviousYear  totalSales  Sales  \
entityIdentifier obligorName                                                    
966413912        Stated                       2.571474e+07         0.0    0.0   
                 Stated                       7.547762e+09         0.0    0.0   
                 Stated                       5.831380e+09         0.0    0.0   
                 Stated                       1.450401e+09         0.0    0.0   
                 Stated                       1.578900e+09         0.0    0.0   

                              totalSalesPreviousYear  totalOperatingProfit  \
entityIdentifier obligorName                                                 
966413912        Stated                          0.0                   NaN   
                 Stated                          0.0                   NaN   
                 Stated                          0.0                   NaN   
                 Stated                          0.0                   NaN   
                 Stated                          0.0                   NaN   

                              totalAmortizationAndDepreciation  \
entityIdentifier obligorName                                     
966413912        Stated                                    NaN   
                 Stated                                    NaN   
                 Stated                                    NaN   
                 Stated                                    NaN   
                 Stated                                    NaN   

                              Interest Expense  Net Income  \
entityIdentifier obligorName                                 
966413912        Stated                    0.0         0.0   
                 Stated                    0.0         0.0   
                 Stated                    0.0         0.0   
                 Stated                    0.0         0.0   
                 Stated                    0.0         0.0   

                             primaryIndustryClassification Industry  \
entityIdentifier obligorName                                          
966413912        Stated                             Sector   Sector   
                 Stated                             Sector   Sector   
                 Stated                             Sector   Sector   
                 Stated                             Sector   Sector   
                 Stated                             Sector   Sector   

                              User-Defined Time Period (optional)  isTTC  
entityIdentifier obligorName                                              
966413912        Stated                                       NaN    NaN  
                 Stated                                       NaN    NaN  
                 Stated                                       NaN    NaN  
                 Stated            

In [213]:
Australias.rename(index={0:'obligorName', 1:'entityIdentifier'})

,Obligor Name,Obligor Key,Datadate,Statement Date - Month,Cash,Current Date - Month,Cash,Total Assets,Accounts Payable,Current Liabilities,Total Liabilities,Prev Yr Total Liabilities,Sales,Prev Yr Sales,Gross Profit,Depreciation and Amortization - ADJUSTMENTSTORECONCILENETINCOME,Interest Expense,Net Income,Industry Classification,Industry,User-Defined Time Period (optional),Financial Statement Only EDF,Risk Calc Model 2.0
69,Stated,999983093,2019-12-31,12,2.814548e+03,06,2.814548e+03,1.553538e+07,0.0,8.882380e+02,8.882380e+02,NaN,0.0,NaN,0.0,NaN,0.0,0.0,Sector,Sector,NaN,NaN,United States
72,Stated,685057921,2015-05-31,5,1.831000e+05,06,1.831000e+05,1.754220e+10,0.0,3.310200e+06,1.293220e+07,NaN,0.0,NaN,0.0,592300.0,0.0,0.0,Sector,Sector,NaN,NaN,United States
77,Stated,596321812,2016-09-30,9,1.600000e+05,06,1.600000e+05,2.494000e+09,0.0,7.570000e+05,2.680000e+06,NaN,0.0,NaN,0.0,67000.0,0.0,0.0,Sector,Sector,NaN,NaN,United States
78,Stated,596321812,2020-09-27,9,3.150000e+05,06,3.150000e+05,2.884000e+09,0.0,7.260000e+05,2.389000e+06,NaN,0.0,NaN,0.0,NaN,0.0,0.0,Sector,Sector,NaN,NaN,United States
81,Stated,596321812,2017-10-01,10,8.800000e+04,06,8.800000e+04,2.782000e+09,0.0,1.182000e+06,2.485000e+06,NaN,0.0,NaN,0.0,75000.0,0.0,0.0,Sector,Sector,NaN,NaN,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8784,Stated,1000054154,2019-12-31,12,3.152293e+07,06,3.152293e+07,5.963500e+11,0.0,1.728270e+08,4.395320e+08,1.708860e+11,0.0,0.0,0.0,NaN,0.0,0.0,Sector,Sector,NaN,NaN,Emerging Markets
8785,Stated,1000054154,2020-12-31,12,3.762053e+07,06,3.762053e+07,6.263550e+11,0.0,2.284010e+08,4.590710e+08,1.374720e+11,0.0,0.0,0.0,NaN,0.0,0.0,Sector,Sector,NaN,NaN,Emerging Markets
8786,Stated,1000055514,2020-12-31,12,3.058920e+05,06,3.058920e+05,2.854792e+09,0.0,5.481510e+05,2.381807e+06,4.590710e+11,0.0,0.0,0.0,NaN,0.0,0.0,Sector,Sector,NaN,NaN,Europe Large Firm
8787,Stated,1000055642,2020-12-31,12,3.399000e+06,06,3.399000e+06,5.041230e+11,0.0,1.389570e+08,3.206420e+08,2.381807e+09,0.0,0.0,0.0,NaN,0.0,0.0,Sector,Sector,NaN,NaN,Emerging Markets


In [ ]:
df['purchase_date'].to_numpy().astype('datetime64[M]'

In [ ]:
#remove entires where max statement year is older than 2019
df16=df15[(df15['Statement Date - Year']>=2019)]

In [214]:
with pd.ExcelWriter('consolidated FAW 06242022_.xlsx') as writer:
    dfdivide.to_excel(writer, sheet_name='Consolidated FAW')

In [215]:
with pd.ExcelWriter('Aus and North America 06242022.xlsx') as writer:
    Australia.to_excel(writer, sheet_name='Australia')
    NorthAmerica.to_excel(writer, sheet_name='North America Large Firm')

In [216]:
with pd.ExcelWriter('Mexico, Europe, Emerging and Canada 06242022.xlsx') as writer:
    Mexico.to_excel(writer,sheet_name='Mexico')
    Europe.to_excel(writer, sheet_name='Europe Large Firm')
    Emerging.to_excel(writer, sheet_name='Emerging Markets')
    Canada.to_excel(writer, sheet_name='Canada')

In [238]:
with pd.ExcelWriter('Aus Arranged.xlsx') as writer:
    Australiamax.to_excel(writer, sheet_name='Arrange')